# Getting Started With Apache Sedona

In [ ]:
from sedona.spark import *

config = SedonaContext.builder().getOrCreate()
sedona = SedonaContext.create(config)

## The Spatial DataFrame

In [ ]:
cities_df = sedona.createDataFrame(
    [
        ("San Francisco", -122.4191, 37.7749), 
        ("New York", -74.0060, 40.7128), 
        ("Austin", -97.7431, 30.2672)
    ], 
    ["city", "longitude", "latitude"])

cities_df.show()

In [ ]:
cities_df.printSchema()

## Introduction To Spatial SQL

In [ ]:
cities_df.createOrReplaceTempView("cities")

In [ ]:
cities_df = sedona.sql("""
SELECT *, ST_Point(longitude, latitude) AS geometry
FROM cities
""")

cities_df.show(truncate=False)

In [ ]:
cities_df.printSchema()

In [ ]:
cities_df.createOrReplaceTempView("cities")

In [ ]:
buffer_df = sedona.sql("""
SELECT city, ST_Buffer(geometry, 1000, true) AS geometry
FROM cities
""")

buffer_df.show()

## Working With The DataFrame API

```
SELECT ST_Makeline(collect_list(geometry)) AS geometry
FROM cities
```

In [ ]:
from sedona.sql.st_functions import ST_MakeLine
from pyspark.sql.functions import collect_list, col

route_df = cities_df.select(ST_MakeLine(collect_list(col("geometry"))).alias("geometry"))

route_df.show(truncate=False)

## Visualizing Data

In [ ]:
SedonaKepler.create_map(cities_df, "Cities")

In [ ]:
mapview = SedonaKepler.create_map(cities_df, "Cities")
SedonaKepler.add_df(mapview, buffer_df, "City Buffer")
SedonaKepler.add_df(mapview, route_df, "City Route")
mapview

## Exercises

1. Create a free Wherobots Cloud account. After signing in, create a tiny Sedona runtime and open the notebook environment. Run all cells in the example "First Wherobots Cloud Notebook". 
2. Read the documentation for spatial SQL functions supported in Apache Sedona. Choose a function to manipulate the +geometry+ column of the `cities_df` DataFrame. What is the input to the function? What is the output?
3. Using SedonaKepler, visualize the results of Exercise 2. Can you visualize this data along with the original `cities_df` DataFrame.